# Machine Learning with SQL | Iris dataset

- This notebook teaches you how to train a machine learning model in database
- Oracle Cloud database is used here
- It supports machine learning functionality
- There's no need to create a database on your own
    - Just follow instructions below to configure the connection

In [63]:
print(f'cx_Oracle: {cx_Oracle.__version__}')
print(f'Pandas: {pd.__version__}')

cx_Oracle: 8.0.1
Pandas: 1.0.5


<br><br>

## Wallet configuration
- Oracle Cloud Wallet has been provided for you in the `/wallet` folder
- You'll have to configure the `wallet/sqlnet.ora` file
    - Change `?/network/wallet` to absolute path of your wallet
    
Here's how:

In [1]:
import os

- This is absolute path to the wallet (on my machine):

In [10]:
os.path.abspath('wallet/')

'/Users/dradecic/Library/Mobile Documents/com~apple~CloudDocs/Firma/OReilly/Notebook/wallet'

- The `sqlnet.ora` file looks like this:

In [5]:
!cat wallet/sqlnet.ora

WALLET_LOCATION = (SOURCE = (METHOD = file) (METHOD_DATA = (DIRECTORY="?/network/admin")))
SSL_SERVER_DN_MATCH=yes

- Change *?/network/admin* with the result of `os.path.abspath('wallet/')`

**After replacing:**

In [11]:
!cat wallet/sqlnet.ora

WALLET_LOCATION = (SOURCE = (METHOD = file) (METHOD_DATA = (DIRECTORY="/Users/dradecic/Library/Mobile Documents/com~apple~CloudDocs/Firma/OReilly/Notebook/wallet")))
SSL_SERVER_DN_MATCH=yes

- Once the wallet is configured, you can establish the connection.

<br><br>

## Establish a Database Connection

- The `cx_Oracle` module is used to communicate with Oracle database
- The credentials are provided for you in the code cell below:

In [13]:
import cx_Oracle

wallet_path = os.path.abspath('wallet/')
username = 'ADMIN'
password = '5r0HpCdHfor3z89uWlXGE3oKU1u92e'
tns_name = 'db202012121152_low'

os.environ['TNS_ADMIN'] = wallet_path
conn = cx_Oracle.connect(username, password, tns_name, encoding='UTF-8', nencoding='UTF-8')

- The connection is now established, and you can continue with table creation and data loading

<br><br>

## Create Table

- You'll use `Pandas` throughout the notebook to work with data
- The code cell below load in the Iris dataset to memory:

In [14]:
import pandas as pd

iris = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


- You need to create a table for holding the Iris dataset, and then we need to load data to it. 
- OML (Oracle Machine Learning) requires one column to be used as row ID (sequence)
- The code cell below creates both sequence and the data, and outputs current contents of the `iris_data` table:

In [15]:
create_sequence = '''CREATE SEQUENCE seq_iris'''
create_table = '''
    CREATE TABLE iris_data(
        iris_id      NUMBER DEFAULT seq_iris.NEXTVAL,
        sepal_length NUMBER,
        sepal_width  NUMBER,
        petal_length NUMBER,
        petal_width  NUMBER,
        species      VARCHAR2(16)
    )
'''

cursor = conn.cursor()
cursor.execute(create_sequence)
cursor.execute(create_table)
cursor.close()

db_iris = pd.read_sql('''SELECT * FROM iris_data''', conn)

In [16]:
db_iris.head()

,IRIS_ID,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH,SPECIES


- The table is created, but empty
- You'll learn how to load in data next

<br><br>

## Load Data into Table

- You can load in data into the table with a Python loop
- The `itertuples()` function is used to itereate through the dataset rows
- On each iteration, a single row is loaded
- When all of the rows load, the transactions is commited:

In [17]:
cursor = conn.cursor()

for row in iris.itertuples(index=False):
    stmt = f'''
        INSERT INTO iris_data (
            sepal_length, 
            sepal_width, 
            petal_length, 
            petal_width, 
            species
        ) VALUES (
            {row.sepal_length},
            {row.sepal_width},
            {row.petal_length},
            {row.petal_width},
            '{row.species}'
        )
    '''
    cursor.execute(stmt)

cursor.close()
conn.commit()

- Here's how the table looks like now:

In [18]:
db_iris = pd.read_sql('''SELECT * FROM iris_data''', conn)
db_iris.head()

,IRIS_ID,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH,SPECIES
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


- Everything is ready for machine learning
- The first step is the train/test split

<br><br>

## Train/Test Split

- Oracle likes this step done with two views — one for training data and one for testing data
- You can easily create these with a single PL/SQL block:

In [20]:
stmt = '''
    BEGIN
        EXECUTE IMMEDIATE 
            'CREATE OR REPLACE VIEW 
            iris_train_data AS 
            SELECT * FROM iris_data 
            SAMPLE (75) SEED (42)';
        EXECUTE IMMEDIATE 
            'CREATE OR REPLACE VIEW 
            iris_test_data AS 
            SELECT * FROM iris_data 
            MINUS 
            SELECT * FROM iris_train_data';
    END;
'''

cursor = conn.cursor()
cursor.execute(stmt)
cursor.close()

In [26]:
print(f"#Rows in Train set: {pd.read_sql('''SELECT COUNT(*) FROM iris_train_data''', conn).values[0][0]}")
print(f"#Rows in Test set : {pd.read_sql('''SELECT COUNT(*) FROM iris_test_data''', conn).values[0][0]}")

#Rows in Train set: 119
#Rows in Test set : 31


- A total of 150 rows, 119 in training set, and 31 in the test set
- You can train the model next

<br><br>

## Model Training

- The easiest method for model training is through `DBMS_DATA_MINING` package, with a single procedure execution, and without the need for creating additional settings tables
- You'll use Decision tree algorithm for the job:

In [27]:
training_stmt = '''
    DECLARE 
        v_setlst DBMS_DATA_MINING.SETTING_LIST;
    BEGIN
        v_setlst('PREP_AUTO') := 'ON';
        v_setlst('ALGO_NAME') := 'ALGO_DECISION_TREE';

        DBMS_DATA_MINING.CREATE_MODEL2(
            'iris_clf_model',
            'CLASSIFICATION',
            'SELECT * FROM iris_train_data',
            v_setlst,
            'iris_id',
            'species'
        );
    END;
'''

cursor = conn.cursor()
cursor.execute(training_stmt)
cursor.close()

- The `CREATE_MODEL2` procedure accepts a lot of parameters. Let’s explain the ones we entered:
    - `iris_clf_model` — simply the name of your model. Can be anything
    - `CLASSIFICATION` — type of machine learning task we’re doing. Must be uppercase for some reason
    - `SELECT * FROM iris_train_data` — specifies where the training data is stored
    - `v_setlst` — above declared settings list for our model
    - `iris_id` — name of the sequence type column (each value is unique)
    - `species` — name of the target variable (what we’re trying to predict)
- Executing this block will take a second or two, but once done it’s ready for evaluation!

- The model is now trained, so let's evaluate it next

<br><br>

## Model Evaluation

- The following PL/SQL block applies the model to the test set:

In [28]:
evaluate_stmt = '''
    BEGIN
        DBMS_DATA_MINING.APPLY(
            'iris_clf_model', 
            'iris_test_data', 
            'iris_id', 
            'iris_apply_result'
        );
    END;
'''

cursor = conn.cursor()
cursor.execute(evaluate_stmt)
cursor.close()

- Here are the prediction probabilities:

In [29]:
test_set_predictions = pd.read_sql('''SELECT * FROM iris_apply_result''', conn)
test_set_predictions

,IRIS_ID,PREDICTION,PROBABILITY,COST
0,7,setosa,1.0,0.0
1,7,versicolor,0.0,1.0
2,7,virginica,0.0,1.0
3,10,setosa,1.0,0.0
4,10,versicolor,0.0,1.0
...,...,...,...,...
88,141,setosa,0.0,1.0
89,141,versicolor,0.0,1.0
90,145,virginica,1.0,0.0
91,145,setosa,0.0,1.0


- You'll somehow need to keep only the classification where the prediction probability is the highest
    - Notice there are three rows for a single instance (3 possible species)
- The snippet below does it:

In [53]:
preds = []
for iris_id in test_set_predictions['IRIS_ID'].unique():
    subset = test_set_predictions[test_set_predictions['IRIS_ID'] == iris_id].reset_index()
    predicted = subset['PROBABILITY'].idxmax()
    preds.append({
        'IRIS_ID': iris_id,
        'PREDICTION': subset.iloc[predicted]['PREDICTION']
    })

- Here are the predictions:

In [54]:
preds

[{'IRIS_ID': 7, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 10, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 11, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 13, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 14, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 17, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 18, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 19, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 20, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 36, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 37, 'PREDICTION': 'setosa'},
 {'IRIS_ID': 53, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 61, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 62, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 67, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 71, 'PREDICTION': 'virginica'},
 {'IRIS_ID': 76, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 82, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 89, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 93, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 97, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 99, 'PREDICTION': 'versicolor'},
 {'IRIS_ID': 102, 'PREDICTION': 'v

- To evauate model performance, you'll need access to the test set
- Let's grab it:

In [55]:
test_set = pd.read_sql('''SELECT * FROM iris_test_data''', conn)
test_set.head()

,IRIS_ID,SEPAL_LENGTH,SEPAL_WIDTH,PETAL_LENGTH,PETAL_WIDTH,SPECIES
0,93,5.8,2.6,4.0,1.2,versicolor
1,136,7.7,3.0,6.1,2.3,virginica
2,102,5.8,2.7,5.1,1.9,virginica
3,7,4.6,3.4,1.4,0.3,setosa
4,71,5.9,3.2,4.8,1.8,versicolor


- You'll join test set with predictions on the `IRIS_ID` column next:

In [58]:
preds_df = pd.DataFrame(preds)

eval_df = pd.merge(
    left=test_set[['IRIS_ID', 'SPECIES']],
    left_on='IRIS_ID',
    right=preds_df,
    right_on='IRIS_ID'
)
eval_df.head()

,IRIS_ID,SPECIES,PREDICTION
0,93,versicolor,versicolor
1,136,virginica,virginica
2,102,virginica,virginica
3,7,setosa,setosa
4,71,versicolor,virginica


- From here, it's easy to calcualate if a prediction was correct:

In [60]:
eval_df['CORRECT'] = [1 if species == prediction else 0 for species, prediction in zip(eval_df['SPECIES'], eval_df['PREDICTION'])]

In [61]:
eval_df.head()

,IRIS_ID,SPECIES,PREDICTION,CORRECT
0,93,versicolor,versicolor,1
1,136,virginica,virginica,1
2,102,virginica,virginica,1
3,7,setosa,setosa,1
4,71,versicolor,virginica,0


- And now to calcuate the accuracy, all you need is to divide the sum of correct predictions with the total dataset length:

In [62]:
eval_df['CORRECT'].sum() / len(eval_df)

0.9032258064516129

- The decision tree model resulted in 90% accuracy.